In [1]:
from pathlib import Path

import pandas as pd
from pandas_indexing import extractlevel, isin

In [2]:
base_path = Path(
    "/Users/coroa/Library/CloudStorage/OneDrive-SharedLibraries-IIASA/RESCUE - WP 1/data"
)
data_path = Path("../data")

## Variable definition files

The variable definition file is a CSV or yaml file that needs to contain the `variable`-name, its `sector`, `gas` components and whether it is expected `global` (or regional instead).

Here we generate one based on the cmip6 historical data we have that could be used as a basis but we would want to finetune this by hand.

### Create new variable definition files from historical data

In [3]:
variables = (
    pd.read_csv(
        base_path / "historical/cmip6/history.csv",
        usecols=[3, 4],
    )
    .rename(columns=str.lower)
    .drop_duplicates()
    .set_index("variable")
)

In [4]:
variabledefs = extractlevel(variables, "CEDS+|9+ Sectors|Emissions|{gas}|{sector}|Unharmonized")

Mark variables as global or regional:

Global variables are:
- Variants of fluorinated gases,
- N2O, as well as
- Aggriculture and LUC, and
- Aircraft and International Shipping

In [5]:
variabledefs["global"] = False
variabledefs.loc[
    isin(gas=["F-Gases", "N2O", "C2F6", "HFC", "SF6", "CF4"]), "global"
] = True
variabledefs.loc[
    isin(
        sector=["Aggregate - Agriculture and LUC", "Aircraft", "International Shipping"]
    ),
    "global",
] = True

In [6]:
variabledefs.to_csv(data_path / "variabledefs.csv")